# Portugal Reservoir - Sentinel-1 & Sentinel-2 Dataset Creator
## Google Colab Version

This notebook will:
1. Install required dependencies
2. Download 36 matched Sentinel-1/Sentinel-2 image pairs
3. Save them to your Google Drive or download locally

**Your ROI**: Portugal Reservoir (38.01°N, 7.94°W)
**Time Range**: 2016-2024 (9 years)
**Expected Output**: ~2-5 GB of satellite imagery

## Step 1: Install Dependencies
This will take 1-2 minutes

In [ ]:
!pip install -q planetary-computer pystac-client rasterio shapely xarray matplotlib pandas
print("✓ Dependencies installed successfully!")

## Step 2: Import Libraries

In [ ]:
import planetary_computer as pc
from pystac_client import Client
import pandas as pd
from datetime import datetime, timedelta
import os
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")

## Step 3: Define Helper Functions

In [ ]:
def search_sentinel2(catalog, bbox, start_date='2016-01-01', end_date='2024-12-31', cloud_percentage=5.0):
    """Search Sentinel-2 imagery."""
    print(f"\nSearching Sentinel-2 imagery...")
    print(f"  Date range: {start_date} to {end_date}")
    print(f"  Cloud coverage: <{cloud_percentage}%")
    
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        query={"eo:cloud_cover": {"lt": cloud_percentage}}
    )
    
    items = list(search.items())
    print(f"  Found {len(items)} Sentinel-2 scenes")
    
    return [{
        'id': item.id,
        'datetime': item.datetime.strftime('%Y-%m-%d %H:%M:%S'),
        'date': item.datetime.strftime('%Y-%m-%d'),
        'timestamp': int(item.datetime.timestamp() * 1000),
        'cloud_cover': item.properties.get('eo:cloud_cover', 0),
        'item': item
    } for item in items]


def search_sentinel1(catalog, bbox, start_date='2016-01-01', end_date='2024-12-31'):
    """Search Sentinel-1 SAR imagery."""
    print(f"\nSearching Sentinel-1 imagery...")
    print(f"  Date range: {start_date} to {end_date}")
    
    search = catalog.search(
        collections=["sentinel-1-rtc"],
        bbox=bbox,
        datetime=f"{start_date}/{end_date}"
    )
    
    items = list(search.items())
    print(f"  Found {len(items)} Sentinel-1 scenes")
    
    return [{
        'id': item.id,
        'datetime': item.datetime.strftime('%Y-%m-%d %H:%M:%S'),
        'date': item.datetime.strftime('%Y-%m-%d'),
        'timestamp': int(item.datetime.timestamp() * 1000),
        'orbit_direction': item.properties.get('sat:orbit_state', 'unknown'),
        'item': item
    } for item in items]


def match_temporal_pairs(s1_items, s2_items, max_time_diff_days=3):
    """Match Sentinel-1 and Sentinel-2 images within time threshold."""
    if not s1_items or not s2_items:
        print("Warning: One or both collections are empty")
        return []
    
    print(f"\nMatching temporal pairs (max {max_time_diff_days} days apart)...")
    
    s1_df = pd.DataFrame(s1_items)
    s2_df = pd.DataFrame(s2_items)
    
    s1_df['datetime_obj'] = pd.to_datetime(s1_df['datetime'])
    s2_df['datetime_obj'] = pd.to_datetime(s2_df['datetime'])
    
    s1_df = s1_df.sort_values('datetime_obj').reset_index(drop=True)
    s2_df = s2_df.sort_values('datetime_obj').reset_index(drop=True)
    
    matched_pairs = []
    
    for idx, s2_row in s2_df.iterrows():
        s2_time = s2_row['datetime_obj']
        s1_df['time_diff'] = abs((s1_df['datetime_obj'] - s2_time).dt.total_seconds() / 86400)
        valid_matches = s1_df[s1_df['time_diff'] <= max_time_diff_days]
        
        if not valid_matches.empty:
            closest_match = valid_matches.loc[valid_matches['time_diff'].idxmin()]
            matched_pairs.append({
                's1_id': closest_match['id'],
                's1_date': closest_match['date'],
                's1_datetime': closest_match['datetime'],
                's1_timestamp': int(closest_match['timestamp']),
                's1_orbit': closest_match['orbit_direction'],
                's1_item': closest_match['item'],
                's2_id': s2_row['id'],
                's2_date': s2_row['date'],
                's2_datetime': s2_row['datetime'],
                's2_timestamp': int(s2_row['timestamp']),
                's2_cloud_cover': float(s2_row['cloud_cover']),
                's2_item': s2_row['item'],
                'time_diff_days': float(closest_match['time_diff'])
            })
    
    print(f"  Found {len(matched_pairs)} matched pairs")
    return matched_pairs

print("✓ Helper functions defined!")

## Step 4: Define Your Portugal ROI

In [ ]:
# Your custom Portugal reservoir polygon
roi_coordinates = [
    [-7.937365, 38.022977], [-7.937064, 38.021946], [-7.937, 38.020559],
    [-7.936227, 38.01897], [-7.935197, 38.018227], [-7.933974, 38.017483],
    [-7.935734, 38.016587], [-7.936013, 38.015606], [-7.936013, 38.014947],
    [-7.935026, 38.014491], [-7.934511, 38.014], [-7.935326, 38.012513],
    [-7.935777, 38.011633], [-7.936141, 38.010687], [-7.936893, 38.009875],
    [-7.939661, 38.009655], [-7.941999, 38.01018], [-7.943716, 38.01116],
    [-7.943716, 38.0128], [-7.942901, 38.012783], [-7.941914, 38.012783],
    [-7.941334, 38.012614], [-7.940412, 38.013544], [-7.94024, 38.014626],
    [-7.93906, 38.015353], [-7.938995, 38.016283], [-7.942042, 38.017635],
    [-7.941356, 38.017838], [-7.940519, 38.01777], [-7.939296, 38.017348],
    [-7.938159, 38.018734], [-7.938673, 38.020238], [-7.938867, 38.02176],
    [-7.93803, 38.023044], [-7.937365, 38.022977]
]

# Calculate bounding box
lons = [coord[0] for coord in roi_coordinates]
lats = [coord[1] for coord in roi_coordinates]
bbox = (min(lons), min(lats), max(lons), max(lats))

print(f"✓ ROI defined: Portugal Reservoir")
print(f"  Bounding box: {bbox}")
print(f"  Area: ~1.08 km × 1.49 km")

## Step 5: Create the Dataset
This will fetch metadata for all matched pairs (takes 1-2 minutes)

In [ ]:
print("="*80)
print("CREATING SENTINEL-1 & SENTINEL-2 DATASET")
print("="*80)

# Connect to Microsoft Planetary Computer
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace
)
print("✓ Connected to Microsoft Planetary Computer")

# Search for images
s2_items = search_sentinel2(catalog, bbox, '2016-01-01', '2024-12-31', 5.0)
s1_items = search_sentinel1(catalog, bbox, '2016-01-01', '2024-12-31')

# Match pairs
matched_pairs = match_temporal_pairs(s1_items, s2_items, max_time_diff_days=3)

print(f"\n{'='*80}")
print(f"DATASET CREATED SUCCESSFULLY!")
print(f"{'='*80}")
print(f"Total Sentinel-1 images: {len(s1_items)}")
print(f"Total Sentinel-2 images: {len(s2_items)}")
print(f"Matched pairs: {len(matched_pairs)}")
print(f"{'='*80}")

## Step 6: View Dataset Statistics

In [ ]:
if matched_pairs:
    df = pd.DataFrame([{
        's1_date': p['s1_date'],
        's2_date': p['s2_date'],
        's2_cloud_cover': p['s2_cloud_cover'],
        'time_diff_days': p['time_diff_days']
    } for p in matched_pairs])
    
    print("\nDATASET STATISTICS:")
    print("="*60)
    print(f"Cloud Coverage:")
    print(f"  Min: {df['s2_cloud_cover'].min():.2f}%")
    print(f"  Max: {df['s2_cloud_cover'].max():.2f}%")
    print(f"  Average: {df['s2_cloud_cover'].mean():.2f}%")
    
    print(f"\nTemporal Alignment:")
    print(f"  Min: {df['time_diff_days'].min():.2f} days")
    print(f"  Max: {df['time_diff_days'].max():.2f} days")
    print(f"  Average: {df['time_diff_days'].mean():.2f} days")
    
    print("\nFirst 5 matched pairs:")
    display(df.head())
    
    # Plot statistics
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(df['s2_cloud_cover'], bins=20, edgecolor='black', color='steelblue')
    axes[0].set_xlabel('Cloud Cover (%)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Cloud Coverage Distribution')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].hist(df['time_diff_days'], bins=20, edgecolor='black', color='coral')
    axes[1].set_xlabel('Time Difference (days)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Temporal Difference Distribution')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No matched pairs found!")

## Step 7: Save Metadata to JSON

In [ ]:
# Save metadata
metadata = {
    'source': 'Microsoft Planetary Computer',
    'roi_type': 'Custom Polygon (Portugal Reservoir)',
    'bbox': list(bbox),
    'start_date': '2016-01-01',
    'end_date': '2024-12-31',
    'cloud_percentage_threshold': 5.0,
    'max_time_diff_days': 3,
    'total_s1_images': len(s1_items),
    'total_s2_images': len(s2_items),
    'matched_pairs_count': len(matched_pairs),
    'matched_pairs': [{
        's1_id': p['s1_id'],
        's1_date': p['s1_date'],
        's1_orbit': p['s1_orbit'],
        's2_id': p['s2_id'],
        's2_date': p['s2_date'],
        's2_cloud_cover': p['s2_cloud_cover'],
        'time_diff_days': p['time_diff_days']
    } for p in matched_pairs]
}

with open('portugal_reservoir_dataset.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✓ Metadata saved to: portugal_reservoir_dataset.json")

# Download the JSON file
from google.colab import files
files.download('portugal_reservoir_dataset.json')
print("✓ JSON file downloaded to your computer!")

## Step 8: Download Actual Satellite Images (OPTIONAL)
⚠️ **WARNING**: This will download ~2-5 GB of data!

Uncomment and run the cell below to download the actual images.

In [ ]:
# # UNCOMMENT TO DOWNLOAD IMAGES
# import rasterio
# from rasterio.merge import merge
# import numpy as np

# # How many pairs to download (start with just 3 for testing)
# NUM_PAIRS_TO_DOWNLOAD = 3

# print(f"Downloading first {NUM_PAIRS_TO_DOWNLOAD} pairs...")
# print("This will take several minutes.\n")

# os.makedirs('images', exist_ok=True)

# for i, pair in enumerate(matched_pairs[:NUM_PAIRS_TO_DOWNLOAD]):
#     print(f"\n[{i+1}/{NUM_PAIRS_TO_DOWNLOAD}] Downloading pair {i+1}...")
    
#     # Download Sentinel-2 RGB
#     s2_item = pair['s2_item']
#     s2_file = f"images/pair_{i:03d}_S2_{pair['s2_date']}.tif"
    
#     try:
#         # Get RGB bands
#         bands = ['B04', 'B03', 'B02']  # Red, Green, Blue
#         datasets = [rasterio.open(s2_item.assets[band].href) for band in bands if band in s2_item.assets]
        
#         if datasets:
#             mosaic, out_trans = merge(datasets)
#             out_meta = datasets[0].meta.copy()
#             out_meta.update({
#                 "driver": "GTiff",
#                 "height": mosaic.shape[1],
#                 "width": mosaic.shape[2],
#                 "transform": out_trans,
#                 "count": mosaic.shape[0]
#             })
            
#             with rasterio.open(s2_file, "w", **out_meta) as dest:
#                 dest.write(mosaic)
            
#             for ds in datasets:
#                 ds.close()
            
#             print(f"  ✓ S2 saved: {s2_file}")
#     except Exception as e:
#         print(f"  ✗ S2 error: {e}")
    
#     # Download Sentinel-1 SAR
#     s1_item = pair['s1_item']
#     s1_file = f"images/pair_{i:03d}_S1_{pair['s1_date']}.tif"
    
#     try:
#         bands = ['vh', 'vv']
#         datasets = [rasterio.open(s1_item.assets[band].href) for band in bands if band in s1_item.assets]
        
#         if datasets:
#             mosaic, out_trans = merge(datasets)
#             out_meta = datasets[0].meta.copy()
#             out_meta.update({
#                 "driver": "GTiff",
#                 "height": mosaic.shape[1],
#                 "width": mosaic.shape[2],
#                 "transform": out_trans,
#                 "count": mosaic.shape[0]
#             })
            
#             with rasterio.open(s1_file, "w", **out_meta) as dest:
#                 dest.write(mosaic)
            
#             for ds in datasets:
#                 ds.close()
            
#             print(f"  ✓ S1 saved: {s1_file}")
#     except Exception as e:
#         print(f"  ✗ S1 error: {e}")

# print(f"\n✓ Downloaded {NUM_PAIRS_TO_DOWNLOAD} pairs to ./images/")
# print("✓ You can now download them or mount Google Drive to save them!")

print("Image download cell ready. Uncomment the code above to start downloading.")

## Step 9: Mount Google Drive (Optional)
To save images directly to your Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# print("✓ Google Drive mounted!")
# print("Change output_dir in Step 8 to: '/content/drive/MyDrive/portugal_reservoir_dataset'")

print("Uncomment the code above to mount Google Drive")

## Summary

### What You've Done:
✅ Created a dataset of matched Sentinel-1/Sentinel-2 pairs  
✅ Downloaded metadata JSON file  
✅ Generated statistics and visualizations  

### Next Steps:
1. Uncomment Step 8 to download actual satellite images
2. Mount Google Drive to save images there
3. Process images for your analysis

### Your Dataset:
- **Location**: Portugal Reservoir (38.01°N, 7.94°W)
- **Time Range**: 2016-2024 (9 years)
- **Matched Pairs**: Check the output above
- **Quality**: Low cloud coverage (<5%)
- **Temporal Alignment**: Within 3 days